In [1]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from langgraph.checkpoint.memory import MemorySaver
import time

In [2]:
class CrashState(TypedDict):
  input: str
  step1: str
  step2: str
  step3: str

In [3]:
# 2. Define steps
def step_1(state: CrashState) -> CrashState:
  print("Step 1 executed")
  return {"step1": "done", "input": state["input"]}

def step_2(state: CrashState) -> CrashState:
  print(" Step 2 hanging ... now manually interrupt from the notebook toolbar (STOP button)")
  time.sleep(30) # Simulate long-running hang
  return {"step2": "done"}

def step_3(state: CrashState) -> CrashState:
  print("Step 3 executed")
  return {"step3": "done"}


In [4]:
builder = StateGraph(CrashState)

builder.add_node('step_1', step_1)
builder.add_node('step_2', step_2)
builder.add_node('step_3', step_3)

builder.add_edge(START, 'step_1')
builder.add_edge('step_1', 'step_2')
builder.add_edge('step_2', 'step_3')
builder.add_edge('step_3', END)

checkpointer = MemorySaver()
graph = builder.compile(checkpointer=checkpointer)



In [ ]:
config1 = {'configurable' : {'thread_id': '1'}}
try:
  print("Running, please interupt manually during step_2")
  graph.invoke({'input':'start'}, config= config1)
except KeyboardInterrupt:
  print('Karnel manually interrupted! Crash simulated.')